In [1]:
import pretty_midi
import torch
import os

In [2]:
def melody_to_numpy(fpath="ashover12.mid", unit_time=0.125):
    # 首先取出midi中的音乐信息，将第一个音轨提出来，然后取所有音符元素。
    music = pretty_midi.PrettyMIDI(fpath)
    notes = music.instruments[0].notes

    # 纪录一个时间戳 t和保存向量的列表 roll。
    t = 0.
    roll = list()
    # print(notes[0], notes[-1])

    # 在 Notes里遍历，找出所有音符的音高和长度，同时不放过休止符。
    for note in notes:
        # print(t, note)

        # 两个相邻的音符不是无缝连接，说明休止符存在。计算其相对于最小分辨率 unit_time的相对时长 T，建立一个(T, 130)的矩阵，将第129维置1.
        elapsed_time = note.start - t
        if elapsed_time > 0.:
            steps = torch.zeros((int(round(elapsed_time / unit_time)), 130))
            steps[range(int(round(elapsed_time / unit_time))), 129] += 1.
            roll.append(steps)

        # 如果是无缝连接，那么检查当前音符：
        n_units = int(round((note.end - note.start) / unit_time))
        steps = torch.zeros((n_units, 130))
        steps[0, note.pitch] += 1
        steps[range(1, n_units), 128] += 1

        # 其中除第一列记录pitch外，其他列都记录sustain的128.最后合成为一个矩阵：
        roll.append(steps)
        t = note.end
    return torch.cat(roll, 0)

In [3]:
def array_to_single(array):
    midi_batch = []
    for i in array:
        num = 0
        for j in i:
            if j == 1:
                midi_batch.append(num)
                num = 0
                break
            num += 1
    return midi_batch

In [4]:
if __name__ == "__main__":
    # musicPath = "./output_MTD_min_data/MTD6657_Prokofiev_Op064-10.mid"
    # midiarray = melody_to_numpy(musicPath)
    # midi_batch = array_to_single(midiarray)
    # print(midi_batch)

    musicData = []
    # 填充字符
    padding = 0
    for filepath,dirnames,filenames in os.walk(r'./output_MTD_min_data'):
        for filename in filenames:
            try:
                # print(os.path.join(filepath,filename))
                musicpath = os.path.join(filepath,filename)
                midiarray = melody_to_numpy(musicpath)
                midi_batch = array_to_single(midiarray)
                if(len(midi_batch)<=64):
                    midi_batch += [padding] * (64 - len(midi_batch))
                    musicData.append(midi_batch)
            except Exception as e:
                print(musicpath)
                continue

./output_MTD_min_data\MTD7108_Rimsky_Op035-01.mid
./output_MTD_min_data\MTD7111_Rimsky_Op035-01.mid
./output_MTD_min_data\MTD7112_Rimsky_Op035-02.mid
./output_MTD_min_data\MTD7185_Roussel_Op042-03.mid
./output_MTD_min_data\MTD7227_Saint_CarnavalAnimaux-02.mid
./output_MTD_min_data\MTD7230_Saint_CarnavalAnimaux-06.mid
./output_MTD_min_data\MTD7238_Saint_Op033-02.mid
./output_MTD_min_data\MTD7266_Saint_Op040.mid
./output_MTD_min_data\MTD7282_Saint_Op028.mid
./output_MTD_min_data\MTD7283_Saint_Op028.mid
./output_MTD_min_data\MTD7284_Saint_Op028.mid
./output_MTD_min_data\MTD7391_Satie_Gnossiennes-01.mid
./output_MTD_min_data\MTD7535_Schubert_D0780-01.mid
./output_MTD_min_data\MTD7538_Schubert_D0780-03.mid
./output_MTD_min_data\MTD7594_Schubert_D0810-04.mid
./output_MTD_min_data\MTD7603_Schubert_D0667-02.mid
./output_MTD_min_data\MTD7606_Schubert_D0667-04.mid
./output_MTD_min_data\MTD7607_Schubert_D0667-05.mid
./output_MTD_min_data\MTD7630_Schubert_D0845-02.mid
./output_MTD_min_data\MTD7633

In [5]:
len(musicData)

93

In [6]:
musicData

[[60,
  129,
  62,
  64,
  65,
  67,
  69,
  71,
  72,
  129,
  72,
  128,
  76,
  128,
  79,
  128,
  84,
  128,
  72,
  128,
  83,
  128,
  71,
  128,
  84,
  128,
  72,
  128,
  129,
  129,
  88,
  128,
  86,
  129,
  84,
  83,
  81,
  79,
  78,
  79,
  81,
  129,
  79,
  77,
  76,
  74,
  71,
  73,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [76,
  128,
  128,
  128,
  67,
  128,
  128,
  128,
  76,
  128,
  128,
  128,
  76,
  128,
  128,
  128,
  76,
  128,
  128,
  128,
  76,
  128,
  79,
  128,
  77,
  128,
  128,
  128,
  77,
  128,
  128,
  128,
  77,
  128,
  128,
  128,
  77,
  128,
  128,
  128,
  77,
  128,
  80,
  128,
  78,
  128,
  81,
  128,
  79,
  128,
  128,
  128,
  81,
  128,
  128,
  128,
  79,
  128,
  128,
  128,
  79,
  128,
  128,
  128],
 [53,
  128,
  128,
  128,
  57,
  128,
  128,
  128,
  58,
  128,
  128,
  128,
  55,
  128,
  128,
  128,
  62,
  128,
  128,
  128,
  60,
  128,
  128,
  128,
  58,
  128,
  128,
  1